In [1]:
import pyodbc
from import_function import import_scoring_data, import_data
from features_by_customer_type_2 import customer_type_features
from care_features_2 import care_features
# from features_by_customer_type import customer_type_features
conn = pyodbc.connect(dsn='VerticaProd')
ct='New'
response=['FLAG']
#  AND CUSTOMER_DEFINITION_ADJ='NEW TO TURBOTAX'
care_cont_features, care_bool_features, care_catag_features, care_cont_score_features, care_bool_score_features, care_catag_score_features=care_features(ct)
cont_features,bool_features,catag_features,cont_score_features,bool_score_features,catag_score_features=customer_type_features(ct)
data = "SELECT A.*, CASE WHEN FLAG='FUD' THEN 0 ELSE 1 END AS FLAG FROM CTG_ANALYTICS_WS.SM_RETENTION_MODEL A INNER JOIN CTG_ANALYTICS_WS.SM_DEFECTION_SURVEY_MODEL B ON A.AUTH_ID=B.AUTH_ID where tax_year=2015 union all SELECT A.*, CASE WHEN FLAG='FUD' THEN 0 ELSE 1 END AS FLAG FROM CTG_ANALYTICS_WS.SM_RETENTION_MODEL A INNER JOIN CTG_ANALYTICS_WS.SM_DEFECTION_SURVEY_MODEL_TY15 B ON A.AUTH_ID=B.AUTH_ID where tax_year=2014"
care_data="SELECT DISTINCT  A.AUTH_ID,  TAX_YEAR, AXC_CARE,ENTERING_CONTACT_US_EXPERIENCE,TOTAL_AGENT_INTERACTION_SECONDS,CARE_REFERRER,	FIRST_HS_INTENT,	FIRST_SEARCH_MANUAL_LOCATION_DETAIL FROM CTG_ANALYTICS_WS.SM_CARE_DATA_MINING A INNER JOIN CTG_ANALYTICS_WS.SM_CUSTOMER_RANDOM_SAMPLE B ON A.AUTH_ID=B.AUTH_ID WHERE A.TAX_YEAR=2015 limit 1"
df_no_pca= import_data(data,care_data, cont_features, bool_features, catag_features,care_cont_features, care_bool_features,care_catag_features)


c:\python27\lib\site-packages\pandas\core\frame.py:2762: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)


df_cont done
just_dummies done
df_trans done


In [2]:
from bin_cont_features import bin_pca
j='True'
response='FLAG'

print df_no_pca.dtypes
dummy_pca, length_dict = bin_pca(df_no_pca,response, j)
scoring_list=list(dummy_pca)

FLAG                                         bool
MINDBENDER_FLAG                              bool
CA_REFUND_TRANSFER_FLAG                      bool
REFUND_TRANSFER_FLAG                         bool
CA_AUDIT_DEFENSE_FLAG                        bool
AUDIT_DEFENSE_FLAG                           bool
CA_MAX_FLAG                                  bool
MAX_FLAG                                     bool
PS_FLAG                                      bool
REQUIRED_TAKE_FLAG                           bool
FSCHC_FLAG                                   bool
FSCHCEZ_FLAG                                 bool
FSCHE_FLAG                                   bool
FSCHA_FLAG                                   bool
FSCHD_FLAG                                   bool
FSCHF_FLAG                                   bool
MISC1099_FLAG                                bool
RT_FLAG                                      bool
student_taxpayer                             bool
_taxpayer                                    bool


bin_cont_features.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df_pca[k+m[0]+repr(m[1])][df_pca[k]<=m[1]]=1
bin_cont_features.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df_pca[k+m[0] + repr(m[1])+m[2]+repr(m[3])][(df_pca[k]>m[1])&(df_pca[k]<m[3])] = 1
bin_cont_features.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df_pca[k+m[0] + repr(m[1])][df_pca[k] >= m[1]] = 1


NUM_EXEMPTIONS
{1: ['<=', 1.5], 4: ['>', 1.5], 5: ['> ', 1.5, ' <', 3.5], 6: ['>=', 3.5]}
{1: ['<=', 1.5], 2: ['<=', 0.5], 3: ['> ', 0.5, ' < ', 1.5], 4: ['>', 1.5], 5: ['> ', 1.5, ' <', 3.5], 6: ['>=', 3.5]}
{1: ['<=', 1.5], 2: ['<=', 0.5], 3: ['> ', 0.5, ' < ', 1.5], 4: ['>', 1.5], 5: ['> ', 1.5, ' <', 3.5], 6: ['>=', 3.5]}
NUM_DEPENDENTS
{1: ['<=', 1.5], 4: ['>', 1.5], 5: ['> ', 1.5, ' <', 3.5], 6: ['>=', 3.5]}
{1: ['<=', 1.5], 2: ['<=', 0.5], 3: ['> ', 0.5, ' < ', 1.5], 4: ['>', 1.5], 5: ['> ', 1.5, ' <', 3.5], 6: ['>=', 3.5]}
{1: ['<=', 1.5], 2: ['<=', 0.5], 3: ['> ', 0.5, ' < ', 1.5], 4: ['>', 1.5], 5: ['> ', 1.5, ' <', 3.5], 6: ['>=', 3.5]}
BUS_WILL_FILE_1099
{1: ['<=', 0.5], 2: ['>', 0.5], 3: ['> ', 0.5, ' <', 1.5], 4: ['>=', 1.5]}
{1: ['<=', 0.5], 2: ['>', 0.5], 3: ['> ', 0.5, ' <', 1.5], 4: ['>=', 1.5]}
BUS_REQUIRE_1099
{1: ['<=', 0.5], 2: ['>', 0.5], 3: ['> ', 0.5, ' <', 1.5], 4: ['>=', 1.5]}
{1: ['<=', 0.5], 2: ['>', 0.5], 3: ['> ', 0.5, ' <', 1.5], 4: ['>=', 1.5]}
AMOUNT_U

In [3]:
response='FLAG'
ct='New'
from feature_clustering import feature_clustering
i='pca'

df_no_pca,y, plsca= feature_clustering(dummy_pca,response, i)
print list(df_no_pca)




sample dataframe shape is:  (3749, 768)
{160: 55.005976272544139, 130: 55.373830174878442, 100: 55.743059090802106, 230: 54.629148751190492, 200: 55.390343392239672, 170: 54.922919867027154, 140: 54.938348420259089, 290: 55.186455778730654, 110: 55.60026541880822, 240: 55.166360280546336, 280: 55.178142865881505, 210: 55.517305178304838, 180: 55.628986735168439, 270: 55.485733367412223, 150: 55.286929249632578, 120: 55.498242739092298, 260: 55.157336674617177, 250: 55.852137404932343, 220: 55.591054145062046, 190: 55.516255233190385}
the best # of components is: 250
[u'DMA_AREA_Des Moines-Ames', u'DMA_AREA_Omaha', 'AMOUNT_TAXES_PAID> 5583.0 <12797.5', u'DMA_AREA_Memphis', u'CUSTOMER_TYPE_1st Year Renewer', 'accountant_taxpayer', u'FED_FORM_TYPE_F1040A', u'DMA_AREA_Birmingham (Anniston and Tuscaloosa)', 'AMOUNT_SALARIES_AND_WAGES>=142682.5', u'DMA_AREA_Tampa-Saint Petersburg (Sarasota)', 'COST_PER_CUST_LAG7> 0.14344292879104614 <0.38552606105804443', u'DMA_AREA_Fort Smith-Fayetteville-S

In [4]:
from clustering import obs_clustering
clust='False'
df=df_no_pca
print df
df, df_w_labels, df_w_clust=obs_clustering(df,response,clust)
print df

                 DMA_AREA_Des Moines-Ames DMA_AREA_Omaha  \
AUTH_ID                                                    
2880471                             False          False   
139609599                           False          False   
658999770                           False          False   
1072352625                          False          False   
4697113928720210                    False          False   
4626745205479560                    False          False   
106000696                           False          False   
172345017                           False          False   
431413955                           False          False   
1273482705                          False          False   
4697113918836610                    False          False   
827503860                           False          False   
195243443                           False          False   
347446526                           False          False   
103410430                           Fals

In [5]:
from get_arrays import get_arrays
i='False'
j='False'
dummy_pca=df
response = 'FLAG'
x, y = get_arrays(dummy_pca, response, i, j)
print x.shape
from sklearn.feature_selection import SelectPercentile, f_classif
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
selector = SelectPercentile(f_classif, percentile=20)
selector.fit(x, y)
print selector.pvalues_
df_pvalues=pd.DataFrame(selector.pvalues_,columns=['p_value'], index=list(x))
print df_pvalues
sig_features=df_pvalues[df_pvalues['p_value'] <=0.2]
sig_features.reset_index(['index'], inplace=True)
sig_features.drop(['p_value'], axis=1, inplace=True)
print sig_features


x=x[sig_features['index'].tolist()]
print x.shape



(3749, 167)
[  1.38119260e-01   8.91690370e-01   1.51078141e-05   1.95024634e-01
   9.22807354e-01   5.02970009e-02   8.79846565e-07   7.31546120e-01
   1.36346633e-10   7.40229368e-01   1.27644919e-02   2.12596925e-01
   4.39253675e-03   6.64319721e-01   1.54438156e-01   1.80312499e-01
   9.68928043e-01   1.13481216e-01   8.22497865e-01   3.82907002e-01
   4.21916573e-01   2.27509054e-06   1.80312499e-01   1.05436953e-02
   2.42747946e-01   2.49776513e-01   3.00081073e-04   7.22072627e-01
   7.10423962e-01   6.91806313e-01   1.27644919e-02   1.18984813e-10
   8.81483904e-07   2.58281892e-02   1.83867835e-01   5.81370545e-01
   7.10423962e-01   1.38119260e-01   4.34364796e-10   2.66943885e-02
   1.21638660e-04   4.29901051e-01   5.11835179e-02   2.19071581e-01
   1.92993046e-01   4.03102848e-01   1.95024634e-01   1.99680477e-01
   5.04324181e-01   8.53713598e-01   2.91335207e-01   7.39591638e-01
   4.84341326e-02   3.76083873e-08   9.94374068e-01   7.31546120e-01
   2.61693150e-01   6.

c:\python27\lib\site-packages\ipykernel\__main__.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [6]:
from model_algorithm import algorithm
response = 'FLAG'
models, name = algorithm(x, y, response)

Nearest Neighbors 59.6201977527 0.942206654991 0.910321489002 0.925989672978 0.924261868096 501 33 53 538
Support Vector 60.3346761921 0.712328767123 0.087986463621 0.156626506024 0.524330310462 513 21 539 52
rbf_svc 58.0967728086 0.906178489703 0.670050761421 0.770428015564 0.796635867602 493 41 195 396


c:\python27\lib\site-packages\sklearn\metrics\classification.py:1113: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\python27\lib\site-packages\sklearn\metrics\classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


poly_svc 59.127398835 0.0 0.0 0.0 0.5 534 0 591 0
lin_svc 63.0488464558 0.706611570248 0.289340101523 0.41056422569 0.578190650012 463 71 420 171
Decision_Tree 63.8844638358 0.95652173913 0.893401015228 0.923884514436 0.924228597502 510 24 63 528
Random_Forest 58.5347020857 0.911764705882 0.944162436548 0.927680798005 0.921519420521 480 54 33 558
logistic_regression 62.8688936097 0.69512195122 0.289340101523 0.408602150538 0.57444533166 459 75 420 171
NeuralNetworkLogistic 62.879830935 0.598052851182 0.72758037225 0.656488549618 0.593190935189 245 289 161 430


c:\python27\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:563: ConvergenceWarning: Stochastic Optimizer: Maximum iterations reached and the optimization hasn't converged yet.
  % (), ConvergenceWarning)


NeuralNetwork 58.1896427449 0.780523255814 0.908629441624 0.839718530102 0.812928953022 383 151 54 537


In [7]:
import pandas as pd
import numpy as np
import pickle
ct='New'
response=['FLAG']
filename = 'finalized_model.sav'
filename2 ='name.sav'
filename3 = 'bin_model.sav'
filename4 = 'feature_selection.sav'

filename5 ='score_model.sav'
from care_features_2 import care_features
from predicting_code import transform_to_pca, bin_pca_score_set, get_scoring_arrays, score_set_feature_selection, predict,get_scoring_arrays
from import_function import import_scoring_data
from features_by_customer_type_2 import customer_type_features
defection=pd.DataFrame()
care_cont_features, care_bool_features, care_catag_features, care_cont_score_features, care_bool_score_features, care_catag_score_features=care_features(ct)
cont_features,bool_features,catag_features,cont_score_features,bool_score_features,catag_score_features=customer_type_features(ct)

# # load the model from disk
# models = pickle.load(open(filename, 'rb'))
# name = pickle.load(open(filename2, 'rb'))
# length_dict = pickle.load(open(filename3, 'rb'))
# plsca = pickle.load(open(filename4, 'rb'))


scoring_data = "SELECT  * FROM CTG_ANALYTICS_WS.SM_RETENTION_MODEL A WHERE TAX_YEAR=2015 and abandoned=1 limit 10000"
care_score_data="SELECT DISTINCT  A.AUTH_ID, TAX_YEAR, AXC_CARE,ENTERING_CONTACT_US_EXPERIENCE,TOTAL_AGENT_INTERACTION_SECONDS,CARE_REFERRER,	FIRST_HS_INTENT,	FIRST_SEARCH_MANUAL_LOCATION_DETAIL FROM CTG_ANALYTICS_WS.SM_CARE_DATA_MINING A INNER JOIN CTG_ANALYTICS_WS.SM_CUSTOMER_RANDOM_SCORE_SAMPLE B ON A.AUTH_ID=B.AUTH_ID WHERE A.TAX_YEAR=2015 limit 1 "

scoring_df=import_scoring_data(scoring_data,care_score_data, cont_score_features, bool_score_features, catag_score_features, care_cont_score_features, care_bool_score_features, care_catag_score_features)

i, j, k=['False','True','pca' ]
print scoring_df
scoring_df.reset_index(['AUTH_ID'], inplace=True)

index=scoring_df['AUTH_ID']
print scoring_df.head()

# scoring_df.drop(['AUTH_ID'], axis=1, inplace=True)
# new_list2=list(set(list(df_no_pca)) & set(list(scoring_df)))
# df_scoring=df_no_pca[new_list2]
# scoring_df.reset_index(['CUSTOMER_KEY'], inplace=True)

# scoring_df.drop(['CUSTOMER_KEY'], axis=1, inplace=True)
# x=scoring_df[scoring_list]
# df_trans_pca = transform_to_pca(scoring_df, fitted_pca, i)
# print df_trans_pca
print length_dict
scoring_df_trans = bin_pca_score_set(scoring_df, length_dict, j)

print scoring_df_trans.shape, x.shape
a=list(x)
b=list(scoring_df_trans)
final=list(set(b).difference(a))
scoring_df_trans.drop(final, axis=1, inplace=True)
print scoring_df_trans.shape
x.drop(list(scoring_df_trans), axis=1, inplace=True)
print x.shape
c=list(x)

m=scoring_df_trans.shape[0]     
n= x.shape[1]
print  scoring_df_trans.shape, x.shape

zeros_df=pd.DataFrame(np.zeros((m, n)), columns=c).astype('bool')



# scoring_df_trans.reset_index(['CUSTOMER_KEY'], inplace=True)
print scoring_df_trans

final_scoring_df=pd.concat([zeros_df,scoring_df_trans,index ], axis=1)
final_scoring_df.set_index('AUTH_ID', inplace=True)
print final_scoring_df.shape

scoring_df_trans=final_scoring_df
print scoring_df_trans.shape
x_score, index=score_set_feature_selection(scoring_df_trans,plsca, k)

# x_score.set_index('CUSTOMER_KEY', inplace=True)
print list(x_score)
x_score.drop(['AUTH_ID'], axis=1, inplace=True)
df_p=predict(models,name,x_score, index)
defection=pd.concat([df_p, defection], axis=1)
print defection.head()
defection.to_csv(path_or_buf='defection_model_1.txt', index=True)


import_function.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df_bool.drop([f], axis=1, inplace=True)


                 MINDBENDER_FLAG REFUND_TRANSFER_FLAG CA_MAX_FLAG MAX_FLAG  \
AUTH_ID                                                                      
4626745176926565           False                False       False    False   
61686453                   False                False       False    False   
70246167                   False                False       False    False   
76077768                   False                False       False    False   
699914700                  False                False       False    False   
4626745197792405           False                 True       False    False   
4626745196969920           False                False       False    False   
70672757                   False                False       False    False   
460667780                  False                False       False    False   
5227198                    False                False       False     True   
118291476                  False                False       Fals

In [8]:
from sklearn.model_selection import learning_curve
from sklearn.ensemble import RandomForestClassifier
import numpy as np

train_sizes, train_scores, valid_scores = learning_curve(RandomForestClassifier(criterion='entropy', n_estimators=20, random_state=np.random.RandomState(0)), x, y,train_sizes=[0.001, 0.01, 0.1, 0.5],  scoring='roc_auc', cv=5)
print train_sizes
for i in train_scores:
    print np.mean(i)
for K in valid_scores:
    print np.mean(K)

[   2   29  299 1499]
1.0
0.989903846154
0.920332693378
0.855833809675
0.544581502128
0.562958737902
0.550889829762
0.563319156707


In [9]:
import pandas as pd
from data_intersection import scoring_data_intersection
y_df=df_no_pca[response]
print scoring_df.head()

data, _ = scoring_data_intersection(df_no_pca, scoring_df)
print data.shape
print scoring_df.head()
df_pca=pd.concat([y_df, data], axis=1)
# df_pca.reset_index(['CUSTOMER_KEY'], inplace=True)
print df_pca.shape


  MINDBENDER_FLAG REFUND_TRANSFER_FLAG CA_MAX_FLAG MAX_FLAG PS_FLAG  \
0           False                False       False    False   False   
1           False                False       False    False   False   
2           False                False       False    False   False   
3           False                False       False    False   False   
4           False                False       False    False   False   

  REQUIRED_TAKE_FLAG FSCHC_FLAG FSCHCEZ_FLAG FSCHE_FLAG FSCHA_FLAG  \
0              False      False        False      False      False   
1              False      False        False      False       True   
2              False      False        False      False      False   
3              False      False        False      False       True   
4              False      False        False      False      False   

          ...         DMA_AREA_West Palm Beach-Fort Pierce  \
0         ...                                        False   
1         ...               